In [1]:
# 1. Imports & Setup
import time
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, START, END
from typing import Annotated

# 2. Define Graph State
class GraphState(TypedDict):
    query: str
    legal: dict
    compliance: dict
    finance: dict
    operations: dict
    legal_memory: List[dict]
    compliance_memory: List[dict]
    finance_memory: List[dict]
    operations_memory: List[dict]

# 3. Define Agent Nodes
def legal_node(state: GraphState):
    start = time.time()
    entry = {
        "agent": "legal",
        "output": state["legal"],
        "timestamp": time.time(),
        "contract_id": "CONTRACT_001"
    }
    print(f"⏱ Legal Agent Time: {time.time() - start:.2f}s")
    return {"legal_memory": [entry]}

def compliance_node(state: GraphState):
    start = time.time()
    entry = {
        "agent": "compliance",
        "output": state["compliance"],
        "timestamp": time.time(),
        "contract_id": "CONTRACT_001"
    }
    print(f"⏱ Compliance Agent Time: {time.time() - start:.2f}s")
    return {"compliance_memory": [entry]}

def finance_node(state: GraphState):
    start = time.time()
    entry = {
        "agent": "finance",
        "output": state["finance"],
        "timestamp": time.time(),
        "contract_id": "CONTRACT_001"
    }
    print(f"⏱ Finance Agent Time: {time.time() - start:.2f}s")
    return {"finance_memory": [entry]}

def operations_node(state: GraphState):
    start = time.time()
    entry = {
        "agent": "operations",
        "output": state["operations"],
        "timestamp": time.time(),
        "contract_id": "CONTRACT_001"
    }
    print(f"⏱ Operations Agent Time: {time.time() - start:.2f}s")
    return {"operations_memory": [entry]}
# 4. Build LangGraph
graph = StateGraph(GraphState)
graph.add_node("legal", legal_node)
graph.add_node("compliance", compliance_node)
graph.add_node("finance", finance_node)
graph.add_node("operations", operations_node)

# 5. Define Parallel Execution
graph.add_edge(START, "legal")
graph.add_edge(START, "compliance")
graph.add_edge(START, "finance")
graph.add_edge(START, "operations")

graph.add_edge("legal", END)
graph.add_edge("compliance", END)
graph.add_edge("finance", END)
graph.add_edge("operations", END)

# 6. Compile Graph
parallel_app = graph.compile()

# 7. Run Parallel Execution
import json

with open("outputs/legal_agent_output.json", "r") as f:
    legal_output = json.load(f)["output"]

with open("outputs/compliance_agent_output.json", "r") as f:
    compliance_output = json.load(f)["output"]

with open("outputs/finance_agent_output.json", "r") as f:
    finance_output = json.load(f)["output"]

with open("outputs/operations_agent_output.json", "r") as f:
    operations_output = json.load(f)["output"]

print("✓ All agent outputs loaded successfully")

print(legal_output["risk_level"])
print(compliance_output["risk_level"])
print(finance_output["risk_level"])
print(operations_output["risk_level"])

input_state = {
    "query": "Review termination, GDPR, payment terms and SLAs",
    "legal": legal_output,
    "compliance": compliance_output,
    "finance": finance_output,
    "operations": operations_output,
    "legal_memory": [],
    "compliance_memory": [],
    "finance_memory": [],
    "operations_memory": []
}

# 8. Verify Parallel Outputs
start_seq = time.time()
legal_node(input_state)
compliance_node(input_state)
finance_node(input_state)
operations_node(input_state)
end_seq = time.time()
print(f"Sequential runtime: {end_seq - start_seq:.2f}s")

start_par = time.time()
parallel_app.invoke(input_state)
end_par = time.time()
print(f"Parallel runtime: {end_par - start_par:.2f}s")

✓ All agent outputs loaded successfully
low
high
medium
medium
⏱ Legal Agent Time: 0.00s
⏱ Compliance Agent Time: 0.00s
⏱ Finance Agent Time: 0.00s
⏱ Operations Agent Time: 0.00s
Sequential runtime: 0.00s
⏱ Compliance Agent Time: 0.00s
⏱ Finance Agent Time: 0.00s
⏱ Legal Agent Time: 0.00s
⏱ Operations Agent Time: 0.00s
Parallel runtime: 0.00s


In [2]:
!pip install faiss-cpu sentence-transformers
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


In [3]:
texts = []
metadata = []

def collect_memory(mem_list):
    for item in mem_list:
        texts.append(json.dumps(item["output"]))
        metadata.append({
            "agent": item["agent"],
            "risk_level": item["output"].get("risk_level", "unknown"),
            "timestamp": item["timestamp"],
            "contract_id": item["contract_id"]
        })

collect_memory(input_state["legal_memory"])
collect_memory(input_state["compliance_memory"])
collect_memory(input_state["finance_memory"])
collect_memory(input_state["operations_memory"])

print("Texts count:", len(texts))

if texts:
    print("Sample text:", texts[0])
else:
    print(" No memory collected")


Texts count: 0
 No memory collected


In [4]:
# ===============================
# COMPLETE FAISS MEMORY CELL
# ===============================

import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# -------------------------------
# 1️⃣ CREATE TEXTS + METADATA
# -------------------------------
texts = [
    "legal risk medium termination clause",
    "compliance risk high GDPR violation",
    "finance risk medium late payment penalties",
    "operations risk low SLA timelines"
]

metadata = [
    {"agent": "legal", "risk_level": "medium"},
    {"agent": "compliance", "risk_level": "high"},
    {"agent": "finance", "risk_level": "medium"},
    {"agent": "operations", "risk_level": "low"},
]

print("✓ Texts created:", len(texts))

# -------------------------------
# 2️⃣ LOAD EMBEDDING MODEL
# -------------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# -------------------------------
# 3️⃣ CREATE EMBEDDINGS
# -------------------------------
embeddings = embedder.encode(texts, convert_to_numpy=True)

if embeddings.ndim == 1:
    embeddings = embeddings.reshape(1, -1)

print("✓ Embeddings shape:", embeddings.shape)

# -------------------------------
# 4️⃣ BUILD FAISS INDEX
# -------------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("✓ FAISS index created with vectors:", index.ntotal)

# -------------------------------
# 5️⃣ QUERY FAISS
# -------------------------------
query = "high risk clauses"
query_embedding = embedder.encode([query], convert_to_numpy=True)

D, I = index.search(query_embedding, k=3)

print("\n Search Results:")
for idx in I[0]:
    print("-----")
    print("Agent:", metadata[idx]["agent"])
    print("Risk:", metadata[idx]["risk_level"])
    print("Text:", texts[idx])


✓ Texts created: 4
✓ Embeddings shape: (4, 384)
✓ FAISS index created with vectors: 4

 Search Results:
-----
Agent: legal
Risk: medium
Text: legal risk medium termination clause
-----
Agent: compliance
Risk: high
Text: compliance risk high GDPR violation
-----
Agent: finance
Risk: medium
Text: finance risk medium late payment penalties


In [5]:
import json
import numpy as np


In [6]:
def query_agent_memory(
    query: str,
    agent: str | None = None,
    top_k: int = 5
):
    """
    Query FAISS vector memory.
    If agent is None → search across all agents.
    """
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, k=top_k)

    results = []
    for idx in I[0]:
        record = {
            "text": texts[idx],
            "metadata": metadata[idx]
        }
        if agent is None or record["metadata"]["agent"] == agent:
            results.append(record)

    return results


In [7]:
legal_memories = query_agent_memory(
    query="termination breach jurisdiction",
    agent="legal",
    top_k=3
)

print(" Legal Memory Results")
for r in legal_memories:
    print(r["metadata"]["risk_level"], "-", r["metadata"]["agent"])


 Legal Memory Results
medium - legal


In [8]:
finance_memories = query_agent_memory(
    query="payment penalty late fee",
    agent="finance",
    top_k=3
)

print("\n Finance Memory Results")
for r in finance_memories:
    print(r["metadata"]["risk_level"], "-", r["metadata"]["agent"])



 Finance Memory Results
medium - finance


In [9]:
all_agent_memory = query_agent_memory(
    query="high risk clauses",
    agent=None,      #  no filter
    top_k=10
)

print("\n All-Agent Memory Results")
for r in all_agent_memory:
    print(
        f"{r['metadata']['agent']} → risk: {r['metadata']['risk_level']}"
    )



 All-Agent Memory Results
legal → risk: medium
compliance → risk: high
finance → risk: medium
operations → risk: low
operations → risk: low
operations → risk: low
operations → risk: low
operations → risk: low
operations → risk: low
operations → risk: low


In [10]:
risk_summary = {}

for r in all_agent_memory:
    agent = r["metadata"]["agent"]
    risk = r["metadata"]["risk_level"]
    risk_summary.setdefault(agent, set()).add(risk)

print("\n Risk Comparison Across Agents")
for agent, risks in risk_summary.items():
    print(f"{agent}: {list(risks)}")



 Risk Comparison Across Agents
legal: ['medium']
compliance: ['high']
finance: ['medium']
operations: ['low']


In [11]:
shared_context = "\n".join([
    f"{r['metadata']['agent']} risk: {r['metadata']['risk_level']}"
    for r in all_agent_memory
])

print("\n Shared Context:")
print(shared_context)



 Shared Context:
legal risk: medium
compliance risk: high
finance risk: medium
operations risk: low
operations risk: low
operations risk: low
operations risk: low
operations risk: low
operations risk: low
operations risk: low


In [12]:
def refine_legal_risk(shared_context: str):
    if "finance risk: high" in shared_context:
        return {
            "risk_level": "high",
            "reason": "Termination clause combined with financial penalties"
        }
    return {
        "risk_level": "medium",
        "reason": "No cross-agent escalation detected"
    }

refined_legal = refine_legal_risk(shared_context)

print("\n Refined Legal Output")
print(json.dumps(refined_legal, indent=2))



 Refined Legal Output
{
  "risk_level": "medium",
  "reason": "No cross-agent escalation detected"
}


In [13]:
metadata.append({
    "agent": "legal",
    "risk_level": refined_legal["risk_level"],
    "timestamp": time.time(),
    "contract_id": "CONTRACT_001",
    "refined": True
})

texts.append(json.dumps(refined_legal))

print("✓ Legal memory updated with refined risk")


✓ Legal memory updated with refined risk


In [14]:
finance_risk_levels = [
    r["metadata"]["risk_level"]
    for r in finance_memories
]

def refine_compliance(finance_risks):
    if "high" in finance_risks:
        return {
            "risk_level": "high",
            "reason": "Financial penalties may trigger compliance violations"
        }
    return {
        "risk_level": "medium",
        "reason": "No financial escalation impact"
    }

refined_compliance = refine_compliance(finance_risk_levels)

print("\n Refined Compliance Output")
print(json.dumps(refined_compliance, indent=2))



 Refined Compliance Output
{
  "risk_level": "medium",
  "reason": "No financial escalation impact"
}


In [15]:
from datetime import datetime

FINAL_CONTRACT_SCHEMA = {
    "contract_id": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "overall_risk": "",
    "average_confidence": 0.0,
    "high_risk_clauses": [],
    "generated_at": ""
}


In [16]:
def get_latest_agent_output(agent_name, metadata):
    """
    Returns the latest stored output for a given agent
    """
    agent_records = [m for m in metadata if m["agent"] == agent_name]
    if not agent_records:
        return None
    return agent_records[-1]


In [17]:
legal_out = get_latest_agent_output("legal", metadata)
compliance_out = get_latest_agent_output("compliance", metadata)
finance_out = get_latest_agent_output("finance", metadata)
operations_out = get_latest_agent_output("operations", metadata)


In [18]:
RISK_PRIORITY = {"low": 1, "medium": 2, "high": 3}

def compute_overall_risk(outputs):
    risks = [o["risk_level"] for o in outputs if o]
    return max(risks, key=lambda r: RISK_PRIORITY.get(r, 0))


In [19]:
def average_confidence(outputs):
    scores = [o.get("confidence", 0.0) for o in outputs if o]
    return round(sum(scores) / len(scores), 2) if scores else 0.0


In [20]:
def collect_high_risk_clauses(outputs):
    clauses = []
    for o in outputs:
        if o and o["risk_level"] == "high":
            clauses.extend(o.get("extracted_clauses", []))
    return clauses


In [21]:
from datetime import datetime

# Step 1: Map outputs correctly
legal_out = legal_output
compliance_out = compliance_output
finance_out = finance_output
operations_out = operations_output

# Step 2: Final schema
FINAL_CONTRACT_SCHEMA = {
    "contract_id": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "overall_risk": "",
    "average_confidence": 0.0,
    "high_risk_clauses": [],
    "generated_at": ""
}

# Step 3: Collect outputs
outputs = [legal_out, compliance_out, finance_out, operations_out]

# Step 4: Build final JSON
final_contract_json = FINAL_CONTRACT_SCHEMA.copy()
final_contract_json.update({
    "contract_id": "CONTRACT_001",
    "legal": legal_out,
    "compliance": compliance_out,
    "finance": finance_out,
    "operations": operations_out,
    "overall_risk": compute_overall_risk(outputs),
    "average_confidence": average_confidence(outputs),
    "high_risk_clauses": collect_high_risk_clauses(outputs),
    "generated_at": datetime.now().isoformat()
})

print(final_contract_json)


{'contract_id': 'CONTRACT_001', 'legal': {'clause_type': 'Legal Analysis', 'extracted_clauses': ["The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.", 'The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination.'], 'risk_level': 'low', 'confidence': 0.85, 'evidence': ['The first sentence of the clause describes different events that can lead to termination.']}, 'compliance': {'clause_type': 'Compliance Analysis', 'extracted_clauses': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."], 'risk_level': 'high', 'confidence': 1.0, 'evidence': ["The clause clearly requires the receiving party to maintain confidentiality of the other party's confidential information."]}, 'finance': {'clause_type'

In [22]:
import json

with open("final_contract_output.json", "w") as f:
    json.dump(final_contract_json, f, indent=2)

print(" Final contract-level JSON saved")


 Final contract-level JSON saved


In [23]:
REPORT_STRUCTURE = [
    "Executive Summary",
    "Overall Risk Assessment",
    "Legal Analysis",
    "Compliance Analysis",
    "Financial Analysis",
    "Operational Analysis",
    "Conclusion & Recommendations"
]


In [24]:
def executive_summary(data):
    return f"""
Overall contract risk is **{data['overall_risk'].upper()}**.

Key observations:
- Multiple clauses increase risk exposure.
- High-risk clauses mainly impact financial or legal areas.
- Immediate review is recommended before approval.
"""


In [25]:
def bullet_section(title, output):
    if not output:
        return f"{title}:\n- No data available\n"

    bullets = "\n".join([f"- {c}" for c in output.get("extracted_clauses", [])])
    return f"""
{title} (Risk: {output['risk_level']}):
{bullets}
"""


In [26]:
def build_report(data):
    report = []

    report.append("### Executive Summary")
    report.append(executive_summary(data))

    report.append("### Overall Risk Assessment")
    report.append(f"- Overall Risk Level: {data['overall_risk']}")
    report.append(f"- Average Confidence Score: {data['average_confidence']}")

    report.append(bullet_section("Legal Analysis", data["legal"]))
    report.append(bullet_section("Compliance Analysis", data["compliance"]))
    report.append(bullet_section("Financial Analysis", data["finance"]))
    report.append(bullet_section("Operational Analysis", data["operations"]))

    report.append("### Conclusion & Recommendations")
    report.append("- Review all high-risk clauses")
    report.append("- Seek legal approval before signing")

    return "\n".join(report)


In [27]:
final_report = build_report(final_contract_json)

with open("final_contract_report.txt", "w") as f:
    f.write(final_report)

print(" Human-readable report generated")


 Human-readable report generated
